# Imports

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import json
from datetime import date
import multiprocessing as mp

from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,IntegerType, StructType,StringType
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql import functions as sf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, when, count, col , split

# Functions

In [60]:
def detect_year(begin_value, end_value, dict_years, ci_id):
    ls_year = list(range(begin_value.year, end_value.year+1))
    filter_year = pd.Series(ls_year)[ pd.Series(ls_year).isin(list(range(2014,2019))) ]
    for yr in filter_year:
        if ci_id not in dict_years[yr]:
            dict_years[yr].append(ci_id) 
    return dict_years

def remove_empty_ls(dct):
    if ((pd.Series(dct.values()).str.len()==0).sum()>=1):
        bol = pd.Series(dct.values()).str.len() >=1
        dct_filter = dict( list(zip(pd.Series(dct.keys())[bol], pd.Series(dct.values())[bol])) )
        return dct_filter
    else:
        return dct

def get_dct_by_year(dct,yr_value):
    dct_by_yr = dict()
    for comp_id, dct_years in dct.items():
        try:
            dct_by_yr[comp_id] = dct_years[yr_value]
        except Exception as e:
            pass
    return dct_by_yr

def identify_sector(firm_id, dc_by_sector):
    for sector_label, list_id_firms in dc_by_sector.items():
        if firm_id in list_id_firms:
            label_firm = sector_label
            break
    return label_firm
    
def dict_of_firms_with_similar_managers(dc_by_year):
    dc_result = dict()
    ls_exclude = []
    for origin_id, origin_list in dc_by_year.items():
        for end_id, end_list in dc_by_year.items():
            if (end_id not in ls_exclude):
                if (origin_id != end_id):
                    common_ci = set(origin_list) & set(end_list)
                    if len(common_ci) > 0:
                        if (origin_id not in dc_result.keys()) & (end_id not in dc_result.keys()):
                            dc_result[origin_id] = {end_id:len(common_ci)}
                            dc_result[end_id] = {origin_id:len(common_ci)}
                        elif (origin_id not in dc_result.keys()) & (end_id in dc_result.keys()):
                            dc_result[origin_id] = {end_id:len(common_ci)}
                            dc_result[end_id][origin_id] = len(common_ci)
                        elif (origin_id in dc_result.keys()) & (end_id not in dc_result.keys()):
                            dc_result[end_id] = {origin_id:len(common_ci)}
                            dc_result[origin_id][end_id] = len(common_ci)
                        elif (origin_id in dc_result.keys()) & (end_id in dc_result.keys()):
                            dc_result[origin_id][end_id] = len(common_ci)
                            dc_result[end_id][origin_id] = len(common_ci)
        ls_exclude.append(origin_id)
    return dc_result

# Load Dataset

In [5]:
df_manager = pd.read_csv('/home/user/Desktop/files_desktop/DATA_managers/adm2018.csv')

In [6]:
df_manager.head(3)

,anio,expediente,ingresos,activos,patrimonio,utilidad_perdida,cantidad_epleados,fecha_ejecucion,fecha_actualizacion,adm_cedula,administrador,adm_fnombramiento,adm_ccargo,cargo,adm_periodo,adm_ftermino,adm_frmercantil
0,2018,1,142310.91,1337788.57,1295310.23,10245.38,12.0,2019-06-18 02:32:57.053000,2019-06-18 03:17:01.126000,1701093039,LLERENA ALBUJA TERESA,2008-03-29 15:13:33.886000,1,GERENTE GENERAL,4.0,2012-03-03 16:28:21.526000,2008-04-08 00:00:00
1,2018,10000,39118392.90,27482362.90,7449106.56,1530469.95,1858.0,2019-06-18 02:34:47.153000,2019-06-18 03:17:02.703000,1706450655,MONTALVO PAREDES PABLO SANTIAGO,2014-07-29 08:50:29.166000,32,PRESIDENTE DEL DIRECTORIO,3.0,2016-01-04 10:26:39.920000,2014-08-07 00:00:00
2,2018,100009,1275257.93,488437.09,147892.77,585.75,111.0,2019-06-18 02:46:54.700000,2019-06-18 03:17:17.590000,0922066337,DOMINGUEZ ESPINOZA STEPHANIA ELIZABETH,2017-02-15 12:24:44.593000,30,PRESIDENTE,2.0,NaN,2017-02-17 00:00:00


In [7]:
df_manager.isna().sum()

anio                        0
expediente                  0
ingresos                    0
activos                     0
patrimonio                  0
utilidad_perdida            0
cantidad_epleados           0
fecha_ejecucion             0
fecha_actualizacion         0
adm_cedula                  0
administrador             780
adm_fnombramiento           0
adm_ccargo                  0
cargo                       0
adm_periodo              2276
adm_ftermino           117444
adm_frmercantil            25
dtype: int64

# Fix columns format

In [8]:
df_manager["adm_ftermino"].fillna("2019-06-18", inplace=True)
df_manager['date_begin'] = pd.to_datetime((df_manager.adm_fnombramiento.str.split().str[0]), infer_datetime_format=True)
df_manager['date_end'] = pd.to_datetime((df_manager.adm_ftermino.str.split().str[0]), infer_datetime_format=True)
df_manager['expediente'] = df_manager.expediente.astype(str)
df_manager['adm_cedula' ]= df_manager.adm_cedula.str.strip()

# Filter dataset

- Part1: Records between the years 2014 -2018
- Part2: Records of those who started working outside the range of years (2014-2018) and finished their work activities within the range of years analyzed.
- Part3: Records of those who started working between the range of years (2014-2018) and until now they are working (2019).
- Part4: Records of those who started working outside the range of years (2014-2018) and until now they are working (2019).

In [9]:

part1 = df_manager[  ((df_manager.date_end < '2019-01-01') & (df_manager.date_end >= '2014-01-01')) & 
((df_manager.date_begin < '2019-01-01') & (df_manager.date_begin >= '2014-01-01'))]
part2 = df_manager[  ((df_manager.date_end < '2019-01-01') & (df_manager.date_end >= '2014-01-01')) & 
( (df_manager.date_begin < '2014-01-01') )]

part3 = df_manager[  ( (df_manager.date_end == '2019-06-18') ) & 
((df_manager.date_begin < '2019-01-01') & (df_manager.date_begin >= '2014-01-01'))]
part4 = df_manager[  ( (df_manager.date_end == '2019-06-18') ) & 
( (df_manager.date_begin < '2014-01-01') )]

df_manager_final = pd.concat([part1, part2, part3, part4 ], ignore_index=True)

# Load Data of companies

In [16]:
# dk_firms = dd.read_csv('/home/user/Desktop/files_desktop/DATA_managers/BASE.csv')
# dk_firms = dk_firms.rename(columns={'Año':'anio'})
# dk_firms.columns = pd.Series(dk_firms.columns).str.lower()

# # filter columns
# only_firm_by_sector_dk = ((dk_firms[['expediente','sector']]).drop_duplicates(subset=['expediente']))

In [10]:
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "20g") \
    .appName('MyFirstCSVLoad') \
    .getOrCreate()
dk_firms = spark.read.format("csv").load("/home/user/Desktop/files_desktop/DATA_managers/BASE.csv", header = 'true')

22/05/12 16:55:33 WARN Utils: Your hostname, user-Inspiron-5570 resolves to a loopback address: 127.0.1.1; using 192.168.200.10 instead (on interface wlp2s0)
22/05/12 16:55:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/12 16:55:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
only_firm_by_sector_spk = (dk_firms.select(col('expediente'),col('rama_actividad')))\
    .dropDuplicates(['expediente'])

In [12]:
only_firm_by_sector_df = only_firm_by_sector_spk.toPandas()

# Update datasets with the same *expediente ids* for both dataset (records of administrators and finance data)

In [13]:
df_manager_update = df_manager_final[df_manager_final.expediente.isin(only_firm_by_sector_df.expediente)]

df_rama_exped_id = only_firm_by_sector_df[only_firm_by_sector_df.expediente.isin(df_manager_update.expediente)]\
    .groupby(['rama_actividad'])\
        .apply( lambda x: list(set(x['expediente'])) )\
            .to_frame('expediente_id')\
                .reset_index()

# Form dictionary with *rama_actividad* as keys and a list of firms.
## E.g
```
{'A': ['159855',
  '9064',
  '301751',
  '170521',
  '709746'],
...,
 'T': ['148299', '6693'],
 'U': ['175145'],
 'Z': ['166147', '166095', '159545', '700555']
}
```

In [14]:
dict_rama_exped_id = dict(list(zip(list(df_rama_exped_id.rama_actividad), list(df_rama_exped_id.expediente_id))))
dict_rama_exped_id = {sector_lab: [ company_id+'-'+sector_lab for company_id in list_firms_ids] for sector_lab, list_firms_ids in dict_rama_exped_id.items() }
# save dict as json file
with open('/home/user/Desktop/files_desktop/DATA_managers/dict_rama_exped_id.json', 'w',encoding='utf-8') as f:
    json.dump(dict_rama_exped_id, f, ensure_ascii=False, indent=4)

# df_final.expediente.drop_duplicates().isin(only_firm_by_sector_df.expediente).sum() # 60580, 60538
# Opening JSON file
f = open('/home/user/Desktop/files_desktop/DATA_managers/dict_rama_exped_id.json')
# returns JSON object as a dictionary
data = json.load(f)

In [15]:
df_manager_update = df_manager_update.merge(only_firm_by_sector_df, how='left', on='expediente')
df_manager_update['new_expediente_rama'] = df_manager_update.expediente +'-'+ df_manager_update.rama_actividad
dict_firm_manager_by_year = dict()

for i,row in df_manager_update.iterrows():
    if row['new_expediente_rama'] not in dict_firm_manager_by_year.keys():
        anio_dict = { 2014:[],2015:[],2016:[],2017:[],2018:[]}
        dict_results_yr = detect_year(row['date_begin'], row['date_end'], anio_dict, row['adm_cedula']  )
        dict_firm_manager_by_year[row['new_expediente_rama']] = dict_results_yr
    else:
        anio_dict = dict_firm_manager_by_year[row['new_expediente_rama']]
        dict_results_yr = detect_year(row['date_begin'], row['date_end'], anio_dict, row['adm_cedula'] )
        dict_firm_manager_by_year[row['new_expediente_rama']] = dict_results_yr


/tmp/ipykernel_12498/2379984788.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  filter_year = pd.Series(ls_year)[ pd.Series(ls_year).isin(list(range(2014,2019))) ]


# Remove empty list by year

In [16]:
dict_firm_manager_by_year_update = {firm_id: remove_empty_ls(dct_yr) for firm_id, dct_yr in dict_firm_manager_by_year.items() }

# Form dict of firm by year to estimate common managers

In [17]:
manager_by_firms_2014,manager_by_firms_2015,manager_by_firms_2016,manager_by_firms_2017,manager_by_firms_2018 =  [ get_dct_by_year(dict_firm_manager_by_year_update, i) for i in range(2014,2019)]

In [20]:
# copy_2014_dict = manager_by_firms_2014.copy()
print("Number of processors: ", mp.cpu_count())

Number of processors:  8


In [19]:

a1 = {'10000-N': ['170645065599', '1704446804999'],
 '10003-C': ['1706563689', '1702090166', '1705324976','0907171615'],
 '100138-C': ['0908881006', '1304323981', '0911221174', '0900427519','0907171615'],
 '100235-G': ['0907171615', '0908911407', '0900919226','1706563689'],
 '100253-G': ['0909779761', '1801667757','0907171615','1706563689'],
 '100330-E': ['0915765655', '1200903829','0907171615'],
 '100390-G': ['0915932222', '0924310527', '0904268794'],
 '100538-F': ['1715246045', '1715271274','0907171615','0907171615','1706563689']}

a2 = {'10098-G': ['1711874667', '1002223236','0910457563','1707347777'],
 '101400-K': ['0908027949', '0908956386','0904582392','0910457563'],
 '101601-A': ['0910457563', '0907663579','1711874667','1707347777'],
 '101612-L': ['1300263017', '0904010162','0903159796','1711874667'],
 '10212-G': ['1701983866', '1707347777', '1702385988','1711874667','0910457563'] }
 
a3 = {'103100-G': ['0911822203', '0915047104','0800270654'],
 '10311-G': ['1709825754', '1708438716','0911822203','0800270654','0904907730'],
 '103393-L': ['0904907730','0911822203'],
 '103456-C': ['0800270654', '0901248328', '0901321745','0904907730']}

In [62]:
%time
p1 = dict_of_firms_with_similar_managers(manager_by_firms_2014)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


KeyboardInterrupt: 

In [61]:
dict_of_firms_with_similar_managers(a1)

{'10003-C': {'100138-C': 1,
  '100235-G': 2,
  '100253-G': 2,
  '100330-E': 1,
  '100538-F': 2},
 '100138-C': {'10003-C': 1,
  '100235-G': 1,
  '100253-G': 1,
  '100330-E': 1,
  '100538-F': 1},
 '100235-G': {'10003-C': 2,
  '100138-C': 1,
  '100253-G': 2,
  '100330-E': 1,
  '100538-F': 2},
 '100253-G': {'10003-C': 2,
  '100138-C': 1,
  '100235-G': 2,
  '100330-E': 1,
  '100538-F': 2},
 '100330-E': {'10003-C': 1,
  '100138-C': 1,
  '100235-G': 1,
  '100253-G': 1,
  '100538-F': 1},
 '100538-F': {'10003-C': 2,
  '100138-C': 1,
  '100235-G': 2,
  '100253-G': 2,
  '100330-E': 1}}

In [63]:
# dict_common_managers_2014 = dict_of_firms_with_similar_managers(manager_by_firms_2014)

import multiprocessing as mp
ts_ls = [a1, a2, a3]

In [234]:
%time
# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

# Step 2: `pool.apply` the `howmany_within_range()`
results = pool.map( dict_of_firms_with_similar_managers , [mdc for mdc in ts_ls])

# Step 3: Don't forget to close
pool.close()    



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [64]:
%time
pool = mp.pool.ThreadPool(1)
t2 = pool.map( lambda mini: dict_of_firms_with_similar_managers(mini) , [manager_by_firms_2014])

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs
